In [140]:
from swagger_parser import SwaggerParser

parser = SwaggerParser(swagger_path='swagger.json')  # Init with file

# Get an example of dict for the definition Foo
parser.definitions_example.get('Pet')

{'id': 42,
 'category': {'id': 42, 'name': 'string'},
 'name': 'doggie',
 'photoUrls': ['string', 'string2'],
 'tags': [{'id': 42, 'name': 'string'}],
 'status': 'available'}

In [141]:
import json

with open("swagger.json","r") as infile:
    swagger = json.load(infile)

In [142]:
from faker import Faker
from chance import chance
import random
fake = Faker()


In [386]:
property_generators = {
    "name":lambda: fake.name(),
    "date-time": lambda: fake.date_time(),
    "username":lambda: chance.email(),
    "email": lambda: chance.email(),
    "id":lambda: fake.uuid4(),
    "quantity":lambda: random.randint(0,100),
    "int":lambda: random.randint(0,100),
    "integer":lambda: random.randint(0,100),
    "bool": lambda: chance.boolean(),
    "boolean": lambda: chance.boolean(),
    "firstName": lambda: fake.name().split(" ")[0],
    "lastName": lambda: fake.name().split(" ")[-1],
    "password": lambda: fake.password(),
    "phone": lambda: fake.phone_number(),
    "url": lambda: chance.url(),
    "photoUrls": lambda: chance.url(),
    "status": lambda: "dead" # this is malformed
}



In [387]:
def generate_from_schema(schema_string):
    return lambda: list(generate(schema_string.split("/")[-1]))[0]

def generate_array_from_schema(schema_string):
    return lambda: [generate_from_schema(schema_string)() for x in range(random.randint(0,10))]

def generate_from_enum(enum):
    return lambda: [random.choice(enum)]

def generate_array_from_enum(enum):
    return lambda: [generate_from_enum(enum)() for x in range(random.randint(0,10))]

def generate_prop_from_name(name):
    return lambda: property_generators[name]() if name in property_generators else None

def generate_array_from_name(name):
    return lambda: [generate_prop_from_name(name)() for x in range(random.randint(0,10))]

def generate_prop_from_type(typ):
    return lambda:  property_generators[typ]() 

def generate_array_from_type(typ):
    return lambda:  [property_generators[typ]()]


In [388]:
def get_parameter_generator_from_definition_property(prop_name, prop_val):
    if "$ref" in prop_val:
        return generate_from_schema(prop_val["$ref"])
    elif prop_val["type"] == "array" and "enum" in prop_val["items"]:
        return generate_array_from_enum(prop_val["items"]["enum"])
    elif prop_val["type"] == "array" and "type" in prop_val["items"] and prop_val["items"]["type"] == "string":
        return generate_array_from_name(prop_name)
    elif prop_val["type"] == "array" and "type" in prop_val["items"] and prop_val["items"]["type"] == "integer":
        return generate_array_from_type(prop_val["items"]["type"])
    elif prop_val["type"] == "array" and "$ref" in prop_val["items"]:
        return generate_array_from_schema(prop_val["items"]["$ref"])
    elif prop_val["type"] == "string" and "enum" in prop_val:
        return generate_from_enum(prop_val["enum"])
    elif prop_val["type"] == "string":
        return generate_prop_from_name(prop_name)
    elif prop_val["type"] == "int" or prop_val["type"] == "integer":
        return generate_prop_from_type(prop_val["type"])
    else:
        raise Exception(prop_name)

def get_command_parameter_generator_from_method_param(param):
    if "schema" in param:
        if "$ref" in param["schema"]:
            return generate_from_schema(param["schema"]["$ref"])
        elif param["schema"]["type"] == "array":
            return generate_array_from_schema(param["schema"]["items"]["$ref"])
        else: 
            raise Exception()
    elif "$ref" in param:
        return generate_from_schema(param["$ref"])
    elif param["type"] == "array" and "enum" in param["items"]:
        return generate_array_from_enum(param["items"]["enum"])
    elif param["type"] == "array" and param["items"]["type"] == "string":
        return generate_array_from_name(param["name"])
    elif param["type"] == "string" and "enum" in param:
        return generate_from_enum(param["enum"])
    elif param["type"] == "string":
        return generate_prop_from_name(param["name"])
    elif param["type"] == "int":
        return generate_prop_from_type(param["type"])

In [389]:
object_generators = {}

def get_generator_for_definition(definition_name):
    definition = swagger['definitions'][definition_name] 
    def object_generator():
        instance = {}       
        # iterate through properties 
        for prop_name in definition['properties']:
            val_gen = get_parameter_generator_from_definition_property(prop_name, definition['properties'][prop_name])
            instance[prop_name] = val_gen() if val_gen is not None else None
        yield instance
    return object_generator
    
for definition_name in swagger["definitions"]:
    object_generators[definition_name] = get_generator_for_definition(definition_name)

def generate(obj_name):
    if obj_name not in object_generators:
        raise Exception("Object " + obj_name + " is unknown")
    return object_generators[obj_name]()

list(generate("Pet"))

[{'id': 64,
  'category': {'id': 24, 'name': 'Scott Johnson'},
  'name': 'Melissa Farmer',
  'photoUrls': ['vurih.edu/sagni/gagsoc/pedguzmi/zikzigu/gozen/matuzhaz',
   'kuhma.io/fake/volmut/huba/jakeh',
   'nizhazpo.ru/monatjol/jasul/jimel/hasvevik/wicmifvom'],
  'tags': [{'id': 91, 'name': 'Kristin Coleman'},
   {'id': 86, 'name': 'Bryan Chambers'}],
  'status': ['available']}]

id
category
id
name


KeyError: 'name'

In [310]:


commands = {}
for path_name in swagger["paths"]:
    path = swagger["paths"][path_name]
    for method in path:
        command = path[method]["summary"] if len(path[method]["summary"]) > 0 else path[method]["description"]
        for param in path[method]["parameters"]:
            if param["in"] == "header":
                continue
                
            commands[command] = get_command_parameter_generator_from_method_param(param)

commands

{'Add a new pet to the store': <function __main__.generate_from_schema.<locals>.<lambda>()>,
 'Update an existing pet': <function __main__.generate_from_schema.<locals>.<lambda>()>,
 'Finds Pets by status': <function __main__.generate_array_from_enum.<locals>.<lambda>()>,
 'Finds Pets by tags': <function __main__.generate_array_from_name.<locals>.<lambda>()>,
 'Updates a pet in the store with form data': <function __main__.generate_prop_from_name.<locals>.<lambda>()>,
 'uploads an image': <function __main__.generate_prop_from_name.<locals>.<lambda>()>,
 'Place an order for a pet': <function __main__.generate_from_schema.<locals>.<lambda>()>,
 'Create user': <function __main__.generate_from_schema.<locals>.<lambda>()>,
 'Creates list of users with given input array': <function __main__.generate_array_from_schema.<locals>.<lambda>()>,
 'Logs user into the system': <function __main__.generate_prop_from_name.<locals>.<lambda>()>,
 'Get user by user name': <function __main__.generate_prop_f

In [311]:
for command in commands:
    print(command)
    print(commands[command]() if commands[command] is not None else "NONE")

Add a new pet to the store
id


TypeError: <lambda>() takes 0 positional arguments but 1 was given

In [239]:
import re
from datetime import datetime
import time

def construct_sentence_from_parameters(parameters, sentence):
    if type(parameters) is list:
        for parameter in parameters:
            if type(parameter) is str:
                parameter_name  = re.sub(r"([A-Z]+)", "_\g<1>", parameter).lower()
                sentence += " " + parameter_name + " " + parameter + ","
            else:
                sentence += construct_sentence_from_parameters(parameter,sentence) + ","
    else:        
        for parameter in parameters:
            parameter_name  = re.sub(r"([A-Z]+)", "_\g<1>", parameter).lower()
            if type(parameters[parameter]) is str:
                sentence += " " + parameter_name + " " + parameters[parameter] + ","
            else:
                sentence += construct_sentence_from_parameters(parameters[parameter], sentence)
    return sentence

for command in commands:
    # generate an utterance that contains all of the command and all of the parameters
    parameters = commands[command]()
    sentence = construct_sentence_from_parameters(parameters, "with")
    print(sentence)
    

with id d69e80d2-69d8-4dd1-affd-9896ac062570,with id d69e80d2-69d8-4dd1-affd-9896ac062570,with id d69e80d2-69d8-4dd1-affd-9896ac062570,with id d69e80d2-69d8-4dd1-affd-9896ac062570, id b2672a27-51ca-46e7-b663-38403037c8c8, name Yvonne Hamilton,, name Nathan Cole,with id d69e80d2-69d8-4dd1-affd-9896ac062570,with id d69e80d2-69d8-4dd1-affd-9896ac062570,with id d69e80d2-69d8-4dd1-affd-9896ac062570,with id d69e80d2-69d8-4dd1-affd-9896ac062570, id b2672a27-51ca-46e7-b663-38403037c8c8, name Yvonne Hamilton,, name Nathan Cole,with id d69e80d2-69d8-4dd1-affd-9896ac062570,with id d69e80d2-69d8-4dd1-affd-9896ac062570,with id d69e80d2-69d8-4dd1-affd-9896ac062570,with id d69e80d2-69d8-4dd1-affd-9896ac062570, id b2672a27-51ca-46e7-b663-38403037c8c8, name Yvonne Hamilton,, name Nathan Cole, photo_urls0 photoUrls0, photo_urls1 photoUrls1, photo_urls2 photoUrls2, photo_urls3 photoUrls3, photo_urls4 photoUrls4, photo_urls5 photoUrls5,with id d69e80d2-69d8-4dd1-affd-9896ac062570,with id d69e80d2-69d8-4dd

TypeError: 'int' object is not iterable

In [201]:
sentence

'withCarl Cochran,'